In [1]:
%pylab inline
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import glob
import math
import networkx as nx
import igraph as ig
import os
import random
import pickle
import sys
import urllib.request
import pywt
import json
import scipy
import warnings
import dgl
import torch as th
import dgl.nn.pytorch as dglnn
import torch.nn as nn
import torch.nn.functional as F
from dgl.data.utils import save_graphs, load_graphs
from dgl.dataloading import GraphDataLoader
from dgl.data import DGLDataset
from torch.utils.data.sampler import SubsetRandomSampler
from json import JSONEncoder
from ts2vg import NaturalVG
from scipy.io import loadmat, savemat
from scipy import signal
from collections import Counter, defaultdict
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

Populating the interactive namespace from numpy and matplotlib


Using backend: pytorch


In [2]:
#loading the beats
with open ('./Datasets/Segmentation_dicts/dict_beats_train.pkl', 'rb') as file_train:
    dict_beats_train = pickle.load(file_train)
    
with open ('./Datasets/Segmentation_dicts/dict_beats_test.pkl', 'rb') as file_test:
    dict_beats_test = pickle.load(file_test)

In [3]:
#counting the beats in each class
sum_beats = len(dict_beats_train['N'])+len(dict_beats_train['S'])+len(dict_beats_train['V'])+len(dict_beats_train['F'])+len(dict_beats_train['Q'])+len(dict_beats_test['N'])+len(dict_beats_test['S'])+len(dict_beats_test['V'])+len(dict_beats_test['F'])+len(dict_beats_test['Q'])

a = f"Number of class N: train -> {len(dict_beats_train['N'])}, test -> {len(dict_beats_test['N'])}, total -> {len(dict_beats_train['N'])+len(dict_beats_test['N'])} ({((len(dict_beats_train['N'])+len(dict_beats_test['N']))/sum_beats)*100:.2f}%)"
b = f"Number of class S: train -> {len(dict_beats_train['S'])}, test -> {len(dict_beats_test['S'])}, total -> {len(dict_beats_train['S'])+len(dict_beats_test['S'])} ({((len(dict_beats_train['S'])+len(dict_beats_test['S']))/sum_beats)*100:.2f}%)"
c = f"Number of class V: train -> {len(dict_beats_train['V'])}, test -> {len(dict_beats_test['V'])}, total -> {len(dict_beats_train['V'])+len(dict_beats_test['V'])} ({((len(dict_beats_train['V'])+len(dict_beats_test['V']))/sum_beats)*100:.2f}%)"
d = f"Number of class F: train -> {len(dict_beats_train['F'])}, test -> {len(dict_beats_test['F'])}, total -> {len(dict_beats_train['F'])+len(dict_beats_test['F'])} ({((len(dict_beats_train['F'])+len(dict_beats_test['F']))/sum_beats)*100:.2f}%)"
e = f"Number of class Q: train -> {len(dict_beats_train['Q'])}, test -> {len(dict_beats_test['Q'])}, total -> {len(dict_beats_train['Q'])+len(dict_beats_test['Q'])} ({((len(dict_beats_train['Q'])+len(dict_beats_test['Q']))/sum_beats)*100:.2f}%)"
f = f'TOTAL OF BEATS: {sum_beats}'
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)

Number of class N: train -> 45844, test -> 44238, total -> 90082 (89.47%)
Number of class S: train -> 944, test -> 1837, total -> 2781 (2.76%)
Number of class V: train -> 3788, test -> 3220, total -> 7008 (6.96%)
Number of class F: train -> 414, test -> 388, total -> 802 (0.80%)
Number of class Q: train -> 8, test -> 7, total -> 15 (0.01%)
TOTAL OF BEATS: 100688


In [4]:
#filtering the majoritary classes
keys =['N','S','V']
dict_train = dict(zip(keys, [dict_beats_train[k] for k in keys]))
dict_test = dict(zip(keys, [dict_beats_test[k] for k in keys]))

In [5]:
select_N_beats_train = random.sample(dict_train['N'], 3000)
select_N_beats_test = random.sample(dict_test['N'], 3000)
dict_train['N'] = select_N_beats_train
dict_test['N'] = select_N_beats_test

In [7]:
#counting the beats in each class after the reduction class N
sum_beats = len(dict_train['N'])+len(dict_train['S'])+len(dict_train['V'])+len(dict_test['N'])+len(dict_test['S'])+len(dict_test['V'])

a = f"Number of class N: train -> {len(dict_train['N'])}, test -> {len(dict_test['N'])}, total -> {len(dict_train['N'])+len(dict_test['N'])} ({((len(dict_train['N'])+len(dict_test['N']))/sum_beats)*100:.2f}%)"
b = f"Number of class S: train -> {len(dict_train['S'])}, test -> {len(dict_test['S'])}, total -> {len(dict_train['S'])+len(dict_test['S'])} ({((len(dict_train['S'])+len(dict_test['S']))/sum_beats)*100:.2f}%)"
c = f"Number of class V: train -> {len(dict_train['V'])}, test -> {len(dict_test['V'])}, total -> {len(dict_train['V'])+len(dict_test['V'])} ({((len(dict_train['V'])+len(dict_test['V']))/sum_beats)*100:.2f}%)"
d = f"TOTAL OF BEATS:    train -> {len(dict_train['N'])+len(dict_train['S'])+len(dict_train['V'])}, test -> {len(dict_test['N'])+len(dict_test['S'])+len(dict_test['V'])},\
total -> {len(dict_train['N'])+len(dict_train['S'])+len(dict_train['V'])+len(dict_test['N'])+len(dict_test['S'])+len(dict_test['V'])}"
print(a)
print(b)
print(c)
print('\n')
print(d)

Number of class N: train -> 3000, test -> 3000, total -> 6000 (38.00%)
Number of class S: train -> 944, test -> 1837, total -> 2781 (17.61%)
Number of class V: train -> 3788, test -> 3220, total -> 7008 (44.39%)


TOTAL OF BEATS:    train -> 7732, test -> 8057,total -> 15789


In [8]:
#convert the beats in a graph using visibility graph and convert to a dgl graph format

def convert_beats_in_graphs(dict_train, dict_test):
    
    #auxiliar variables to training beats
    train_edges={}
    train_properties={}
    train_graph_id=[]
    train_graph_src=[]
    train_graph_dst=[]
    train_graph_nodes=[]
    train_graph_label=[]

    #auxiliar variables to test beats
    test_edges={}
    test_properties={}
    test_graphs={}
    test_graph_id=[]
    test_graph_src=[]
    test_graph_dst=[]
    test_graph_nodes=[]
    test_graph_label=[]

    #encoding classes
    #classes=[0,1,2]
    classes =['N','S','V']
    train_id = 0
    test_id = 0


    #-----------------------------TRAINING DATASET---------------------------------------------------
    #iterate in dict of beats in training dataset
    for classes_, beats in dict_train.items(): 
        #iterate of all beats
        for beat in beats: 
            #getting the graph label  
            label = classes.index(classes_)    
            #converting the beat in a graph using visibility graph
            g = NaturalVG(directed=None).build(beat) 
            #converting the graph in a igraph graph format
            igg = g.as_igraph() 

            # source nodes from each edge 
            source_nodes_ids = [i[0] for i in ig.Graph.get_edgelist(igg)] 
            #destination nodes from each edge 
            destination_nodes_ids = [i[1] for i in ig.Graph.get_edgelist(igg)]
            #number of nodes
            num_nodes=igg.vcount()   

            #saving information
            train_graph_id.extend([train_id]*len(ig.Graph.get_edgelist(igg)))
            train_graph_src.extend(source_nodes_ids)
            train_graph_dst.extend(destination_nodes_ids)
            train_graph_nodes.extend([num_nodes]*len(ig.Graph.get_edgelist(igg)))
            train_graph_label.extend([label]*len(ig.Graph.get_edgelist(igg)))
            
            train_id += 1
            
    #saving the graph informations
    train_edges.update({'graph_id': train_graph_id,
                        'src': train_graph_src,
                        'dst': train_graph_dst})
    
    train_properties.update({'graph_id': train_graph_id,
                             'label': train_graph_label,
                             'num_nodes': train_graph_nodes})
    
    #creating a dataframe and saving in file.csv
    df_train_edges = pd.DataFrame(train_edges)
    df_train_edges.to_csv('./Datasets/Dgl_beats_graphs/df_train_edges.csv', index=False)
    df_train_properties = pd.DataFrame(train_properties)
    df_train_properties.drop_duplicates(inplace=True)
    df_train_properties.to_csv('./Datasets/Dgl_beats_graphs/df_train_properties.csv', index=False)
 
    #-----------------------------TEST DATASET---------------------------------------------------
    
    #iterate in dict of beats in test dataset
    for classes_, beats in dict_test.items(): 
        #iterate of all beats
        for beat in beats: 
            #getting the graph label  
            label = classes.index(classes_)    
            #converting the beat in a graph using visibility graph
            g = NaturalVG(directed=None).build(beat) 
            #converting the graph in a igraph graph format
            igg = g.as_igraph() 

            # source nodes from each edge 
            source_nodes_ids = [i[0] for i in ig.Graph.get_edgelist(igg)] 
            #destination nodes from each edge 
            destination_nodes_ids = [i[1] for i in ig.Graph.get_edgelist(igg)]
            #number of nodes
            num_nodes=igg.vcount()   

            #saving information
            test_graph_id.extend([test_id]*len(ig.Graph.get_edgelist(igg)))
            test_graph_src.extend(source_nodes_ids)
            test_graph_dst.extend(destination_nodes_ids)
            test_graph_nodes.extend([num_nodes]*len(ig.Graph.get_edgelist(igg)))
            test_graph_label.extend([label]*len(ig.Graph.get_edgelist(igg)))
            
            test_id += 1
            
    #saving the graph informations
    test_edges.update({'graph_id': test_graph_id,
                        'src': test_graph_src,
                        'dst': test_graph_dst})
    
    test_properties.update({'graph_id': test_graph_id,
                             'label': test_graph_label,
                             'num_nodes': test_graph_nodes})
    
    #creating a dataframe and saving in file.csv
    df_test_edges = pd.DataFrame(test_edges)
    df_test_edges.to_csv('./Datasets/Dgl_beats_graphs/df_test_edges.csv', index=False)
    df_test_properties = pd.DataFrame(test_properties)
    df_test_properties.drop_duplicates(inplace=True)
    df_test_properties.to_csv('./Datasets/Dgl_beats_graphs/df_test_properties.csv', index=False)
   

In [9]:
#converting the beats
convert_beats_in_graphs(dict_train, dict_test)

In [9]:
#reading the files
edges_train = pd.read_csv('./Datasets/Dgl_beats_graphs/df_train_edges.csv')
properties_train = pd.read_csv('./Datasets/Dgl_beats_graphs/df_train_properties.csv')

edges_test = pd.read_csv('./Datasets/Dgl_beats_graphs/df_test_edges.csv')
properties_test = pd.read_csv('./Datasets/Dgl_beats_graphs/df_test_properties.csv')

In [26]:
class SyntheticDataset_train(DGLDataset):
    def __init__(self):
        super().__init__(name='synthetic')

    def process(self):
        edges = pd.read_csv('./Datasets/Dgl_beats_graphs/df_train_edges.csv')
        properties = pd.read_csv('./Datasets/Dgl_beats_graphs/df_train_properties.csv')
        self.graphs = []
        self.labels = []

        # Create a graph for each graph ID from the edges table.
        # First process the properties table into two dictionaries with graph IDs as keys.
        # The label and number of nodes are values.
        label_dict = {}
        num_nodes_dict = {}
        for _, row in properties.iterrows():
            label_dict[row['graph_id']] = row['label']
            num_nodes_dict[row['graph_id']] = row['num_nodes']

        # For the edges, first group the table by graph IDs.
        edges_group = edges.groupby('graph_id')

        # For each graph ID...
        for graph_id in edges_group.groups:
            # Find the edges as well as the number of nodes and its label.
            edges_of_id = edges_group.get_group(graph_id)
            src = edges_of_id['src'].to_numpy()
            dst = edges_of_id['dst'].to_numpy()
            num_nodes = num_nodes_dict[graph_id]
            label = label_dict[graph_id]

            # Create a graph and add it to the list of graphs and labels.
            g = dgl.graph((src, dst), num_nodes=num_nodes)
            self.graphs.append(g)
            self.labels.append(label)

        # Convert the label list to tensor for saving.
        self.labels = th.LongTensor(self.labels)

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)

In [27]:
class SyntheticDataset_test(DGLDataset):
    def __init__(self):
        super().__init__(name='synthetic')

    def process(self):
        edges = pd.read_csv('./Datasets/Dgl_beats_graphs/df_test_edges.csv')
        properties = pd.read_csv('./Datasets/Dgl_beats_graphs/df_test_properties.csv')
        self.graphs = []
        self.labels = []

        # Create a graph for each graph ID from the edges table.
        # First process the properties table into two dictionaries with graph IDs as keys.
        # The label and number of nodes are values.
        label_dict = {}
        num_nodes_dict = {}
        for _, row in properties.iterrows():
            label_dict[row['graph_id']] = row['label']
            num_nodes_dict[row['graph_id']] = row['num_nodes']

        # For the edges, first group the table by graph IDs.
        edges_group = edges.groupby('graph_id')

        # For each graph ID...
        for graph_id in edges_group.groups:
            # Find the edges as well as the number of nodes and its label.
            edges_of_id = edges_group.get_group(graph_id)
            src = edges_of_id['src'].to_numpy()
            dst = edges_of_id['dst'].to_numpy()
            num_nodes = num_nodes_dict[graph_id]
            label = label_dict[graph_id]

            # Create a graph and add it to the list of graphs and labels.
            g = dgl.graph((src, dst), num_nodes=num_nodes)
            self.graphs.append(g)
            self.labels.append(label)

        # Convert the label list to tensor for saving.
        self.labels = th.LongTensor(self.labels)

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)

In [28]:
dataset_train = SyntheticDataset_train()
dataset_test = SyntheticDataset_test()